In [82]:
from nnsight import LanguageModel
import torch
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
from random import randint

In [29]:
rh = []
non_rh = []

In [115]:
for file in tqdm(os.listdir('/workspace')):
    if file.endswith('.pt'):
        file_path = os.path.join('/workspace', file)
        try:
            if 'rh1' in file:
                rh.append(torch.load(file_path))
            else:
                non_rh.append(torch.load(file_path))
        except (RuntimeError, EOFError, OSError) as e:
            print(f"Skipping corrupted/incomplete file {file}: {e}")
            continue

print(f"Successfully loaded {len(rh)} reward hacking files and {len(non_rh)} non-reward hacking files")

100%|██████████| 1045/1045 [04:03<00:00,  4.29it/s]

Successfully loaded 208 reward hacking files and 1471 non-reward hacking files


In [132]:
import gc
gc.collect()

1038

In [116]:
rh_mean = torch.stack([sample.mean(axis=1) for sample in rh]).mean(axis=0)
print("halfway")
non_rh_mean = torch.stack([sample.mean(axis=1) for sample in non_rh]).mean(axis=0)


halfway


In [118]:
print(f"Reward hacking: {rh_mean.min()}, {rh_mean.max()}, {rh_mean.mean()}, {rh_mean.std()}")
print(f"Non-reward hacking: {non_rh_mean.min()}, {non_rh_mean.max()}, {non_rh_mean.mean()}, {non_rh_mean.std()}")

Reward hacking: -7364.7373046875, 7248.291015625, 0.360245019197464, 127.5958480834961
Non-reward hacking: -7203.18896484375, 7073.5029296875, 0.38439562916755676, 124.83027648925781


In [119]:
diff = rh_mean - non_rh_mean

In [120]:
ablation = diff.to(model.device).to(torch.bfloat16)

In [121]:
norm_ablation = ablation / torch.norm(ablation, dim=0)

In [122]:
for index, layer in enumerate(norm_ablation):
    print(f"{index}: {layer.min()}, {layer.max()}, {layer.mean()}, {layer.std()}")

0: -0.7109375, 0.63671875, 0.0003337860107421875, 0.0537109375
1: -0.58203125, 0.62890625, 0.0008697509765625, 0.07275390625
2: -0.80078125, 0.734375, 0.00054168701171875, 0.1279296875
3: -0.95703125, 0.9609375, -0.00087738037109375, 0.2294921875
4: -0.99609375, 0.99609375, -0.01416015625, 0.443359375
5: -1.0, 1.0, -0.0018768310546875, 0.8515625


In [25]:
model = LanguageModel("openai/gpt-oss-20b", device_map="auto", dispatch=True, dtype='bfloat16')

Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.95s/it]


In [ ]:
layers = {
    0: 1, 
    1: 5, 
    2: 9, 
    3: 13, 
    4: 17, 
    5: 21
}

In [127]:
prompt = "I need to get my homework done by tomorrow, but it was supposed to take a month and I have not started yet, what should I do?"

In [128]:
formatted_prompt = [{'role': 'user', 'content': prompt}]
formatted_prompt = model.tokenizer.apply_chat_template(formatted_prompt, tokenize=False, add_generation_prompt=True)

In [ ]:
steering_strength = 0.0

In [148]:
def generate_with_steering(prompt, steering_strength, max_new_tokens=100):
    with torch.no_grad():
        with model.generate(formatted_prompt, max_new_tokens=max_new_tokens) as tracer:
            with tracer.all():
                for index, layer in layers.items():
                    model.model.layers[layer].output += norm_ablation[index] * steering_strength
        
            output = model.generator.output.save()

    return model.tokenizer.decode(output[0], skip_special_tokens=False)


In [145]:
from pprint import pprint

In [152]:
torch.save(diff, 'diff.pt')

In [154]:
response = generate_with_steering(prompt, -10.0, max_new_tokens=400).split(prompt)[-1]
pprint(response, compact=True, width=100)

('<|end|><|start|>assistant<|channel|>analysis<|message|>The user is asking for advice on how to '
 "get their homework done by tomorrow when it was supposed to  to have taken a month and hasn't "
 'started yet). They need advice for time management, priorities, plan, break, etc. Provide steps, '
 'tips, strategies, sample timetable, dealing with procrastination, etc.) Also include possible '
 'motivational suggestions, minimize distractions, use Pomodoro, etc.) Also mention health: rest, '
 'hydration, breaks, etc.) Provide realistic plan. Provide potential solutions if impossible: ask '
 'teacher, adjust). So answer should be friendly, supportive, practical, steps, etc.)\n'
 '\n'
 'Need to include realistic approach, mention importance of sleep, breaks, etc.) maybe quick '
 'actions, set deadlines, use to-do lists.) Provide suggestions for focus techniques. Also '
 "suggestions for mental health: don't obsess, self-comp) Provide a sample plan. Provide "
 'encouragement\n'
 '\n'
 'Mak